In [1]:
import pyspark

import pandas as pd

In [2]:
# first let us try to import the data with pandas

import time

start_time = time.time()

df=pd.read_csv(r'C:\nyc-taxi-dataset\combined\2019-NYC-taxi-data.csv')

df.head()

print("--- %s seconds ---" % (time.time() - start_time))


C:\Users\arprakash\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


--- 642.2118389606476 seconds ---


In [3]:
# that's around six minutes, but pandas did read it wow!

# eleven minutes the second time, first time i restarted so all memory cache was cleared and hence faster? maybe

df.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,0,1.0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,1.0,N,151,239,1.0,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,1.0,2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.6,1.0,N,239,246,1.0,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2.0,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.0,1.0,N,236,236,1.0,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,3,2.0,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,4,2.0,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.0,2.0,N,193,193,2.0,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


In [4]:
# let us see the info of the data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84399019 entries, 0 to 84399018
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   VendorID               float64
 2   tpep_pickup_datetime   object 
 3   tpep_dropoff_datetime  object 
 4   passenger_count        float64
 5   trip_distance          float64
 6   RatecodeID             float64
 7   store_and_fwd_flag     object 
 8   PULocationID           int64  
 9   DOLocationID           int64  
 10  payment_type           float64
 11  fare_amount            float64
 12  extra                  float64
 13  mta_tax                float64
 14  tip_amount             float64
 15  tolls_amount           float64
 16  improvement_surcharge  float64
 17  total_amount           float64
 18  congestion_surcharge   float64
dtypes: float64(13), int64(3), object(3)
memory usage: 11.9+ GB


In [5]:
# now let us try with pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
.master('local[*]') \
.appName('first_spark_application') \
.getOrCreate()

spark

In [9]:
# let us define the schema below and import it to spark.read.csv later

#from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType

#schema = StructType([StructField("vendor_id", IntegerType()),StructField(" pickup_datetime", TimestampType()),
  #                   StructField(" dropoff_datetime",TimestampType()),StructField(" passenger_count", IntegerType()),
    #                 StructField(" trip_distance",DoubleType()),StructField(" pickup_longitude",DoubleType()),
  #                   StructField(" pickup_latitude",DoubleType()),StructField(" rate_code",IntegerType()),
  #                   StructField(" store_and_fwd_flag",StringType()),StructField(" dropoff_longitude",DoubleType()),
  #                   StructField(" dropoff_latitude",DoubleType()),StructField(" payment_type",StringType()),
  #                   StructField(" fare_amount",DoubleType()),StructField(" surcharge",DoubleType()),
  #                   StructField(" mta_tax",DoubleType()),StructField(" tip_amount",DoubleType()),
  #                   StructField(" tolls_amount",DoubleType()),StructField(" total_amount",DoubleType())                 
                     
     #                 ])

#schema

In [6]:
# let us read the data with spark

start_time = time.time()

df_spark = spark.read.csv(r'C:\nyc-taxi-dataset\combined\2019-NYC-taxi-data.csv',header=True, nullValue='NA')

df_spark.show(5)

print("--- %s seconds ---" % (time.time() - start_time))

+---+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|_c0|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+---+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|  0|     1.0| 2019-01-01 00:46:40|  2019-01-01 00:53:20|            1.0|          1.5|       1.0|                 N|         151|         239|         1.0|        7.0|  0.5|    0.5|      1.65|         0.0|     

without inferring schema, spark can read this in 9 seconds wow!

not bad, 45 seconds to evaluate some 84 million rows


In [7]:
# without inferSchema, everything should be string type

# let us verify that

df_spark.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



In [8]:
# let us see if casting works to change data types

from pyspark.sql.functions import to_timestamp

df_spark = df_spark.withColumn('tpep_pickup_datetime',to_timestamp(df_spark['tpep_pickup_datetime']))

df_spark.select('tpep_pickup_datetime').dtypes

[('tpep_pickup_datetime', 'timestamp')]

wow, this method seems to be working

how about we put all the metadata information like this for any columns other than string type

In [10]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType

df_spark = df_spark.withColumn('tpep_dropoff_datetime',to_timestamp(df_spark['tpep_dropoff_datetime']))

df_spark = df_spark.withColumn('passenger_count',df_spark['passenger_count'].cast(IntegerType()))

df_spark = df_spark.withColumn('trip_distance',df_spark['trip_distance'].cast(DoubleType()))  

df_spark = df_spark.withColumn('PULocationID',df_spark['PULocationID'].cast(DoubleType())) 

df_spark = df_spark.withColumn('DOLocationID',df_spark['DOLocationID'].cast(DoubleType())) 

df_spark = df_spark.withColumn('fare_amount',df_spark['fare_amount'].cast(DoubleType())) 

df_spark = df_spark.withColumn('extra',df_spark['extra'].cast(DoubleType())) 

df_spark = df_spark.withColumn('mta_tax',df_spark['mta_tax'].cast(DoubleType())) 

df_spark = df_spark.withColumn('tip_amount',df_spark['tip_amount'].cast(DoubleType())) 

df_spark = df_spark.withColumn('tolls_amount',df_spark['tolls_amount'].cast(DoubleType())) 

df_spark = df_spark.withColumn('improvement_surcharge',df_spark['improvement_surcharge'].cast(DoubleType())) 

df_spark = df_spark.withColumn('total_amount',df_spark['total_amount'].cast(DoubleType())) 

df_spark = df_spark.withColumn('congestion_surcharge',df_spark['congestion_surcharge'].cast(DoubleType())) 

# let us print the schema and verify that everything is complete

df_spark.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: double (nullable = true)
 |-- DOLocationID: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



Yes, it has been executued! Wow, we now know to get back the schema in pyspark instead of wasting team inferring the schema



In [12]:
# how many rows are we exploring

start_time = time.time()

print(df_spark.count())

print("--- %s seconds ---" % (time.time() - start_time))

84399019
--- 68.16249537467957 seconds ---


wow, 84 million rows and spark took 68 seconds to answer that question

In [13]:
# what is the total amount of all taxi fares for year of 2019

start_time = time.time()

print(df_spark.agg({'total_amount':'sum'}).collect()[0][0])

print("--- %s seconds ---" % (time.time() - start_time))

1614077495.766805
--- 175.7431800365448 seconds ---


Wow, that is an incredible $ 1.61 billion spent in taxi fares for a year.

No wonder uber is a unicorn!

Spark has done that calculation for us in three minutes

In [14]:
# how long will pandas take

# let us find out

start_time = time.time()

print (df['total_amount'].sum())

print("--- %s seconds ---" % (time.time() - start_time))


1614077495.7499979
--- 5.7549028396606445 seconds ---


wow, pandas was lightning fast

and probably because it has already got all the data in memory when we loaded it

In [15]:
# how fast will pandas and spark perform on computation

# for e.g., let us add in a column with spark first

start_time = time.time()

df_spark = df_spark.withColumn('total amount less tax',df_spark['total_amount']-df_spark['mta_tax'])

print(df_spark.select('total_amount','mta_tax','total amount less tax').show(5))

print("--- %s seconds ---" % (time.time() - start_time))

+------------+-------+---------------------+
|total_amount|mta_tax|total amount less tax|
+------------+-------+---------------------+
|        9.95|    0.5|                 9.45|
|        16.3|    0.5|                 15.8|
|         5.8|    0.5|                  5.3|
|        7.55|    0.5|                 7.05|
|       55.55|    0.5|                55.05|
+------------+-------+---------------------+
only showing top 5 rows

None
--- 0.6706459522247314 seconds ---


wow, spark does it in less than a second amazing!

In [17]:
# now how about pandas

start_time = time.time()

df['total_amount less tax']=df['total_amount']-df['mta_tax']

print(df.head())

print("--- %s seconds ---" % (time.time() - start_time))

   Unnamed: 0  VendorID tpep_pickup_datetime tpep_dropoff_datetime  \
0           0       1.0  2019-01-01 00:46:40   2019-01-01 00:53:20   
1           1       1.0  2019-01-01 00:59:47   2019-01-01 01:18:59   
2           2       2.0  2018-12-21 13:48:30   2018-12-21 13:52:40   
3           3       2.0  2018-11-28 15:52:25   2018-11-28 15:55:45   
4           4       2.0  2018-11-28 15:56:57   2018-11-28 15:58:33   

   passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0              1.0            1.5         1.0                  N   
1              1.0            2.6         1.0                  N   
2              3.0            0.0         1.0                  N   
3              5.0            0.0         1.0                  N   
4              5.0            0.0         2.0                  N   

   PULocationID  DOLocationID  payment_type  fare_amount  extra  mta_tax  \
0           151           239           1.0          7.0    0.5      0.5   
1           239   

again, a very decent less than a second operation with pandas

In [18]:
# let us see money spend on taxi's by month

# let us first extract the month from the date column

from pyspark.sql.functions import month

start_time = time.time()

df_spark=df_spark.withColumn('Month number',month(df_spark['tpep_pickup_datetime']))

print(df_spark.select('tpep_pickup_datetime','Month number').show(5))

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+------------+
|tpep_pickup_datetime|Month number|
+--------------------+------------+
| 2019-01-01 00:46:40|           1|
| 2019-01-01 00:59:47|           1|
| 2018-12-21 13:48:30|          12|
| 2018-11-28 15:52:25|          11|
| 2018-11-28 15:56:57|          11|
+--------------------+------------+
only showing top 5 rows

None
--- 0.4616222381591797 seconds ---


again, very fast in pyspark wow

In [20]:
# let us group the taxi money by month

start_time = time.time()

df_spark_bymonth = df_spark.groupBy('Month number').sum('total_amount')

print(df_spark_bymonth.show())

print("--- %s seconds ---" % (time.time() - start_time))

+------------+--------------------+
|Month number|   sum(total_amount)|
+------------+--------------------+
|          12| 1.354624075912886E8|
|           1|1.2025571111114666E8|
|           6| 1.370208758012431E8|
|           3|1.5031053158140665E8|
|           5|1.4776838935147017E8|
|           9|1.3081726913124491E8|
|           4|1.4293865640143976E8|
|           8|1.1951251739124504E8|
|           7|1.2376032823118111E8|
|          10|1.4249212155130276E8|
|          11|1.3348042147113205E8|
|           2|1.3025826615111229E8|
+------------+--------------------+

None
--- 309.1172311306 seconds ---


wow, that was intense! took 5 minutes to finish those calculations

In [ ]:
# how will pandas fare?

# first convert the pickup datetime to datetime formats

df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime'])

# did that work

df.info()